In [1]:
!pip install --quiet \
    streamlit \
    pymupdf \
    langchain \
    langchain-community \
    langchain-core \
    langchain-groq \
    langchain-huggingface \
    deep-translator \
    sentence-transformers \
    chromadb \
    pyngrok \
    langdetect \
    huggingface-hub \
    tiktoken \
    pypdf \
    speechrecognition \
    pydub \
    gTTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt install ffmpeg
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%%writefile app.py
import os
import fitz
import streamlit as st
from deep_translator import GoogleTranslator
from langdetect import detect
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

# 🎙️ Voice Input Support
import speech_recognition as sr
from pydub import AudioSegment
import tempfile
from gtts import gTTS
import base64

# 🔊 Text-to-Speech Audio Playback
def play_audio_from_text(text, lang_code="en"):
    tts = gTTS(text=text, lang=lang_code)
    tts.save("response.mp3")
    with open("response.mp3", "rb") as f:
        audio_bytes = f.read()
        b64 = base64.b64encode(audio_bytes).decode()
        md = f"""
        <audio autoplay controls>
        <source src="data:audio/mp3;base64,{b64}" type="audio/mp3">
        </audio>
        """
        st.markdown(md, unsafe_allow_html=True)


# ========== CONFIGURATION ==========
st.set_page_config(page_title="Indian LawBot", layout="wide")
os.environ["GROQ_API_KEY"] = "...................."
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model_name="llama3-8b-8192")

# ========== LAWBOOK PDF PATHS ==========
base_path = "/content/drive/MyDrive/LAW"

lawbooks = {
    "Bharatiya Sakshya Adhiniyam": f"{base_path}/Bharatiya Sakshya Adhiniyam.pdf",
    "Bharatiya Nagarik Suraksha Sanhita": f"{base_path}/Bharatiya_Nagarik_Suraksha_Sanhita.pdf",
    "Civil Procedure Code": f"{base_path}/Civil Procedure Code.pdf",
    "Constitution of India": f"{base_path}/constitution of india.pdf",
    "Bharatiya Nyaya Sanhita": f"{base_path}/Bharatiya_Nyaya_Sanhita.pdf",
    "Consumer Protection Act": f"{base_path}/Consumer Protection Act.pdf",
    "RERA": f"{base_path}/THE REAL ESTATE (REGULATION AND DEVELOPMENT) ACT, 2016.pdf"
}

# 👩 Women-Centric Laws
women_lawbooks = {
    "Dowry Prohibition Act": f"{base_path}/Dowry_prohibition.pdf",
    "Indecent Representation of Women Act, 1986": f"{base_path}/Indecent_representation_of_women_(prohibition)_act,_1986.pdf",
    "Maternity Benefit Act": f"{base_path}/Maternity_benefit.pdf",
    "Maternity Benefit (Amendment) Act, 2017": f"{base_path}/Maternity_benefit_amendment_act2017_.pdf",
    "MTP Amendment Act, 2021": f"{base_path}/MTP Amendment Act 2021.pdf",
    "Protection of Women from Domestic Violence Act, 2005": f"{base_path}/Protection_of_women_from_domestic_violence_act,_2005.pdf",
    "National Commission for Women Act, 1990": f"{base_path}/The National Commission for Women Act, 1990.pdf",
    "POSH Act, 2013": f"{base_path}/The Sexual Harassment of Women at Workplace (Prevention, Prohibition and Redressal) Act, 2013.pdf",
    "Immoral Traffic (Prevention) Act, 1956": f"{base_path}/The_immoral_traffic_(prevention)_act,_1956.pdf",
    "Prohibition of Child Marriage Act, 2006": f"{base_path}/The_prohibition_of_child_marriage_act,_2006.pdf"
}


# 🧒 Child-Centric Laws
child_lawbooks = {
    "Juvenile Justice (Care and Protection of Children) Act, 2015": f"{base_path}/Juvenile Justice (Care and Protection of Children) Act, 2015.pdf",
    "Protection of Children from Sexual Offences (POCSO) Act, 2012": f"{base_path}/Protection of Children from Sexual Offences (POCSO) Act, 2012.pdf",
    "Child Labour (Prohibition and Regulation) Act, 1986 (Amendment 2016)": f"{base_path}/THE CHILD LABOUR (PROHIBITION AND REGULATION) AMENDMENT ACT, 2016.pdf",
    "Right of Children to Free and Compulsory Education Act, 2009": f"{base_path}/The_right_of_children_to_free_and_compulsory_education_act_2009.pdf",
    "Prohibition of Child Marriage Act, 2006": f"{base_path}/The_prohibition_of_child_marriage_act,_2006.pdf",
    "The Commissions for Protection of Child Rights Act, 2005": f"{base_path}/The Commissions for Protection of Child Rights Act, 2005.pdf"
}

# ========== SIDEBAR ==========
st.sidebar.title("⚖️ Indian LawBot")

st.sidebar.markdown("""
### 📘 General Law Books
- Bharatiya Sakshya Adhiniyam
- Bharatiya Nagarik Suraksha Sanhita
- Civil Procedure Code
- Constitution of India
- Bharatiya Nyaya Sanhita
- Consumer Protection Act
- RERA
""")

st.sidebar.markdown("""
### 👩‍⚖️ Women-Centric Laws
- Dowry Prohibition Act
- Indecent Representation of Women Act, 1986
- Maternity Benefit Act
- Maternity Benefit (Amendment) Act, 2017
- MTP Amendment Act, 2021
- Protection of Women from Domestic Violence Act, 2005
- National Commission for Women Act, 1990
- POSH Act, 2013
- Immoral Traffic (Prevention) Act, 1956
- Prohibition of Child Marriage Act, 2006
""")

st.sidebar.markdown("""
### 🧒 Child-Centric Laws
- Juvenile Justice Act, 2015
- Protection of Children from Sexual Offences (POCSO) Act, 2012
- Child Labour (Prohibition and Regulation) Amendment Act, 2016
- Right of Children to Free and Compulsory Education Act, 2009
- Prohibition of Child Marriage Act, 2006
- Commissions for Protection of Child Rights Act, 2005
""")

st.sidebar.markdown("""
---
### 🛠 Quick Actions
- ❓ Ask legal questions
- 📄 Upload documents
- ⚖️ Find the right court

ℹ️ *This assistant simplifies legal language and offers guidance before consulting a lawyer.*
""")

st.sidebar.text_input("Your suggestion:", key="sidebar_suggestion")
st.sidebar.button("Submit", key="sidebar_submit")

# ========== TABS ==========
tab1, tab2, tab3 = st.tabs(["❓ Legal Q&A", "📄 Document Help", "⚖️ Court Finder"])

# ========== TAB 1: Legal Q&A ==========
with tab1:
    st.header("❓ Ask Your Legal Question")
    st.subheader("💬 Enter or 🎙️ Record Your Legal Question")

    # Text input field
    query_input = st.text_input("📝 Type your legal question (any language):", key="qa_query")

    # 🎤 Upload Voice Input (Instead of recording)
    st.markdown("##### 🎤 Or upload your voice question (WAV format)")
    audio_file = st.file_uploader("Upload audio file", type=["wav"])

    voice_query = None

    if audio_file is not None:
        try:
          # Save uploaded file temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
              tmp.write(audio_file.read())
              temp_file_path = tmp.name

          # Force convert to PCM WAV using pydub
            sound = AudioSegment.from_file(temp_file_path)
            pcm_wav_path = temp_file_path.replace(".wav", "_converted.wav")
            sound.export(pcm_wav_path, format="wav", parameters=["-acodec", "pcm_s16le"])  # 16-bit PCM

          # Now process using speech_recognition
            recognizer = sr.Recognizer()
            with sr.AudioFile(pcm_wav_path) as source:
              audio_data = recognizer.record(source)

            try:
              voice_query = recognizer.recognize_google(audio_data)
              st.success(f"🗣️ Transcribed voice: {voice_query}")
            except sr.UnknownValueError:
              st.warning("Could not understand the voice input.")
            except sr.RequestError as e:
              st.error(f"API error during transcription: {e}")

        except Exception as e:
          st.error(f"❌ Audio processing failed: {e}")


    # 👇 Final query: prioritize text, fallback to voice
    final_query = query_input.strip() if query_input.strip() else (voice_query or "").strip()

    if final_query:
        st.markdown(f"**Query Used:** `{final_query}`")
        detected_lang = detect(final_query)
        translated_query = GoogleTranslator(source="auto", target="en").translate(final_query)

        embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectorstore = Chroma(persist_directory=f"{base_path}/law_vector_db", embedding_function=embedder)
        qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(search_kwargs={"k": 5}), return_source_documents=True)

        result = qa_chain(translated_query)
        legal_answer_en = result['result']
        legal_answer_translated = GoogleTranslator(source='en', target=detected_lang).translate(legal_answer_en)

        # 🧠 Generate simplified explanation ahead of mode selection
        prompt = f"""
You're a legal expert. Explain this legal content simply like you're helping someone unfamiliar with legal jargon:
{legal_answer_en}
"""
        simplified = llm.invoke(prompt).content.strip()

        view_mode = st.radio("Choose view mode:", ["🔍 Legal Answer", "✨ Simplified Explanation"], key="qa_view_mode")

        if view_mode == "🔍 Legal Answer":
            st.markdown("### ⚖️ Legal Answer")
            st.success(legal_answer_translated)

            st.markdown("### ✨ Simplified Explanation")
            st.info(GoogleTranslator(source='en', target=detected_lang).translate(simplified))
        else:
            st.markdown("### ✨ Simplified Explanation Only")
            st.info(GoogleTranslator(source='en', target=detected_lang).translate(simplified))
        # 🔈 Speak the Answer
        try:
            spoken_text = legal_answer_translated if view_mode == "🔍 Legal Answer" else GoogleTranslator(source='en', target=detected_lang).translate(simplified)
            lang_code = detect(spoken_text)
            play_audio_from_text(spoken_text, lang_code)
        except Exception as e:
            st.warning(f"Audio playback failed: {e}")

# ========== TAB 2: Document Help ==========
with tab2:
    st.header("📤 Upload and Analyze Legal Document")
    uploaded_file = st.file_uploader("Upload PDF or TXT legal document", type=['pdf', 'txt'], key="doc_upload")

    if uploaded_file:
        ext = uploaded_file.name.split(".")[-1].lower()
        raw_text = ""
        if ext == "pdf":
            doc = fitz.open(stream=uploaded_file.read(), filetype="pdf")
            raw_text = "\n".join([page.get_text() for page in doc])
        elif ext == "txt":
            raw_text = uploaded_file.read().decode("utf-8")

        if not raw_text.strip():
            st.warning("⚠ No text found in the uploaded document.")
        else:
            detected_lang = detect(raw_text[:500])
            action = st.radio("Choose action:", ["Ask Questions", "Summarize Document"], key="doc_action")

            if action == "Ask Questions":
                splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
                chunks = splitter.split_text(raw_text)
                docs = [Document(page_content=chunk) for chunk in chunks]
                embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
                vectordb = Chroma.from_documents(docs, embedding=embedder)
                local_qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever(search_kwargs={"k": 5}), return_source_documents=True)

                user_q = st.text_input("Ask from the document:", key="doc_user_q")
                if user_q:
                    translated_q = GoogleTranslator(source="auto", target="en").translate(user_q)
                    result = local_qa(translated_q)
                    ans_en = result["result"]
                    ans_trans = GoogleTranslator(source="en", target=detected_lang).translate(ans_en)
                    st.success(ans_trans)

            elif action == "Summarize Document":
                short_prompt = f"""
You're a legal assistant. Give a simple, one-paragraph summary of this legal content for a general user.
Avoid legal jargon. Focus on what the document is about, the issue it covers, and what it means for the user.
CONTENT:
{raw_text[:3000]}
"""
                try:
                    response = llm.invoke(short_prompt).content.strip()
                    translated = GoogleTranslator(source="en", target=detected_lang).translate(response)
                    st.markdown("### 📄 Document Summary")
                    st.info(translated.strip())
                except Exception as e:
                    st.error(f"❌ Error during summarization: {e}")

            if st.button("✅ Generate Recommended Steps", key="doc_generate_steps"):
              checklist_prompt = f"""
You are a legal assistant. Read the following legal document.

Your job: Based on what this document says, give the user **3–6 SHORT and RELEVANT steps** they can take before talking to a lawyer.

🎯 Very Important:
- Don’t explain what the document says — that’s already summarized.
- Give real-world, useful, short actions.
- No general suggestions like “research” or “highlight.”
- Only include steps **specific to what happened in this case.**
- Use bullet points, 1-line each, starting with a verb in a simple language.

Document:
{raw_text[:3000]}
"""
              steps = llm.invoke(checklist_prompt).content.strip()
              st.markdown("### 🧾 Recommended Steps Based on Document")
              st.success(GoogleTranslator(source='en', target=detected_lang).translate(steps))

# ========== TAB 3: Court Finder ==========
with tab3:
    st.header("⚖️ Find the Right Court for Your Case")

    with st.form("court_finder_form"):
        case_type = st.selectbox("⚖ Type of Case", ["Criminal", "Civil", "Family", "Consumer", "Labour", "Property", "Appeal", "Other"])
        amount_involved = st.text_input("💰 Claim Amount (for Civil/Consumer only)", placeholder="e.g., 50000")
        special_case = st.selectbox("❗ Special Case Type (if applicable)", ["No", "Public Interest Litigation (PIL)", "Writ Petition", "Appeal in High Court"])
        submitted = st.form_submit_button("🔍 Recommend Court")

    def recommend_court(case_type, amount_involved, special_case):
        if special_case == "Public Interest Litigation (PIL)":
            return "High Court or Supreme Court – File under Article 226 or 32."
        elif special_case == "Writ Petition":
            return "High Court – Filed under Article 226."
        elif special_case == "Appeal in High Court":
            return "High Court – Appeals from District/Tribunal judgments."

        try:
            amount = int(amount_involved)
        except:
            amount = None

        if case_type == "Criminal":
            return "Approach the Judicial Magistrate or Sessions Court based on the IPC section."
        elif case_type == "Civil":
            if amount:
                if amount <= 100000:
                    return "Junior Civil Judge Court (Small Causes Court)."
                elif amount <= 1000000:
                    return "Senior Civil Judge Court (Subordinate Court)."
                else:
                    return "District Civil Court (Principal Civil Court)."
            return "District Civil Court based on nature of dispute."
        elif case_type == "Consumer":
            if amount:
                if amount <= 5000000:
                    return "District Consumer Disputes Redressal Commission (DCDRC)."
                elif amount <= 20000000:
                    return "State Consumer Disputes Redressal Commission (SCDRC)."
                else:
                    return "National Consumer Disputes Redressal Commission (NCDRC)."
            return "Consumer Forum – Specify claim amount."
        elif case_type == "Family":
            return "Family Court in your district – Handles divorce, custody, and maintenance."
        elif case_type == "Labour":
            return "Labour Court or Industrial Tribunal depending on the dispute."
        elif case_type == "Property":
            return "Civil Court or Revenue Court depending on type of property dispute."
        return "Consult a legal expert or your District Civil Court."

    if submitted:
        suggestion = recommend_court(case_type, amount_involved, special_case)
        st.markdown("### 🧭 Recommended Court")
        st.success(suggestion)

# ========== STYLE ==========
st.markdown("""
<style>
div.stAlert > div {
    font-size: 16px;
}
section.main > div {
    padding-top: 15px;
}
</style>
""", unsafe_allow_html=True)


Writing app.py


In [5]:
!streamlit run app.py &>/content/log.txt &

In [ ]:
from pyngrok import conf, ngrok
conf.get_default().auth_token = "..............."
ngrok.kill()
public_url = ngrok.connect(8501)
print("🚀 App is live at:", public_url)

🚀 App is live at: NgrokTunnel: "https://0bda47f780a3.ngrok-free.app" -> "http://localhost:8501"
